In [8]:
# Import modules
import sys
from argparse import ArgumentParser
from pylmm import input
from pylmm.lmm import LMM
from scipy import linalg
import numpy as np
import time

In [2]:
def define_options():
    # Argument parsing
    parser = ArgumentParser(description = 'Calculate variance components')
    parser.add_argument("-b", "--bfile", type = str,
        help = "PLINK binary bed file prefix")
    parser.add_argument("-p", "--pheno", type = str,
        help = "Phenotype file in PLINK format: FID, IID, [PHENOTYPES]")   
    parser.add_argument("-k", "--kinship", type = str,
        help = "Kinship matrix (n x n plain text file)") 
    parser.add_argument("-c", "--covfile", type = str,
        help = "Covariate file in PLINK format: FID, IID, [COVARIATES]")
    parser.add_argument("-o", "--outfile", type = str,
        help = "Output file") 
    parser.add_argument("-v", "--verbose", action = "store_true",
        help = "Print extra info [default=%(default)s]", default = False) 
        ## Other options
        # remove missing genotypes
        # no mean
    return parser
    
parser = define_options()
if len(sys.argv)==1:
    parser.print_help()
    sys.exit(1)
#args = parser.parse_args()

In [3]:
bfile = "/users/rg/dgarrido/tmp/46a85af2c8796fd60a4cf493cc84f7/geno"
pheno = "/users/rg/dgarrido/tmp/46a85af2c8796fd60a4cf493cc84f7/newpheno.txt"
kinship = "/users/rg/dgarrido/tmp/46a85af2c8796fd60a4cf493cc84f7/real1000g.sXX.txt"
outfile = "/users/rg/dgarrido/tmp/46a85af2c8796fd60a4cf493cc84f7/lala.tsv"
covfile = None # "/users/rg/dgarrido/tmp/46a85af2c8796fd60a4cf493cc84f7/covariates.txt"
verbose = True

In [4]:
if verbose: sys.stderr.write("Reading input files...\n")
# Read PLINK input and covariate file
IN = input.plink(bfile, type = 'b', phenoFile = pheno)

if covfile: 
    C = IN.getCovariates(covfile) # Read covariate file, write into input.plink
    X0 = np.hstack([np.ones((IN.phenos.shape[0], 1)), C]) # Added global mean 
    if np.isnan(X0).sum(): 
        parser.error("The covariate file should not contain missing values.")
    if verbose: sys.stderr.write("Read %d covariate(s) from the covariate file\n" % (C.shape[1]))
else: 
    X0 = np.ones((IN.phenos.shape[0], 1)) 
    
if np.isnan(IN.phenos).sum():
    parser.error("The phenotype file should not contain missing values.")

2504


Reading input files...
Read 2504 individuals from /users/rg/dgarrido/tmp/46a85af2c8796fd60a4cf493cc84f7/geno.fam


In [5]:
# Read Kinship: this method seems to be the fastest and works if you already know the size of the matrix
begin = time.time()
if kinship[-3:] == '.gz':
    import gzip
    f = gzip.open(kinship,'r')
    F = f.read() # Might exhaust mem if the file is very large
    K = np.fromstring(F, sep = ' ') # Assume space separated
    f.close()
else: 
    K = np.fromfile(open(kinship, 'r'), sep = ' ')
    K.resize((len(IN.indivs), len(IN.indivs)))
end = time.time()
if verbose: sys.stderr.write("Read the %d x %d kinship matrix in %0.3fs\n" % (K.shape[0], K.shape[1], end-begin))

Read the 2504 x 2504 kinship matrix in 3.737s


In [11]:
# Compute variance components
phenoNum = IN.phenos.shape[1]
sys.stderr.write("Read %d phenotype(s)\n" % phenoNum)
out = open(outfile, 'w')
begin = time.time()
Kva, Kve = linalg.eigh(K)
end = time.time()
if verbose: sys.stderr.write("Kinship eigendecomposition time: %0.3fs\n" % (end - begin))
for i in range(0, phenoNum):
    sys.stderr.write("Phenotype number: %d \n" % (i+1))
    Y = IN.phenos[:,i]
    L = LMM(Y, K, Kva, Kve, X0, verbose = verbose)
    if verbose: sys.stderr.write("Computing fit for null model\n")
    L.fit()
    if verbose: 
        sys.stderr.write("\t heritability=%0.5f, varG=%0.5f, varE=%0.5f\n" % (L.optH,L.optH*L.optSigma, L.optSigma*(1-L.optH)))
        out.write("%0.5f\t%0.5f\n" % (L.optH*L.optSigma, L.optSigma*(1-L.optH)))       
out.close()        

Read 10 phenotype(s)
Kinship eigendecomposition time: 5.207s
Phenotype number: 1 
Cleaning 1 eigenvalues
Computing fit for null model
	 heritability=0.06457, varG=0.04489, varE=0.65028
Phenotype number: 2 
Computing fit for null model
	 heritability=0.04586, varG=0.04405, varE=0.91650
Phenotype number: 3 
Computing fit for null model
	 heritability=0.10655, varG=0.07213, varE=0.60480
Phenotype number: 4 
Computing fit for null model
	 heritability=0.03673, varG=0.06461, varE=1.69472
Phenotype number: 5 
Computing fit for null model
	 heritability=0.18533, varG=0.20033, varE=0.88057
Phenotype number: 6 
Computing fit for null model
	 heritability=0.18787, varG=0.21990, varE=0.95062
Phenotype number: 7 
Computing fit for null model
	 heritability=0.15992, varG=0.07775, varE=0.40843
Phenotype number: 8 
Computing fit for null model
	 heritability=0.13878, varG=0.22391, varE=1.38944
Phenotype number: 9 
Computing fit for null model
	 heritability=0.02099, varG=0.01315, varE=0.61352
Phenoty